In [1]:
!pip install python-docx
!pip install PyPDF2
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.4 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/nanda-03/Job-CVs-matching.git

Cloning into 'Job-CVs-matching'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 2.47 MiB | 8.64 MiB/s, done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import torch
from transformers import CamembertModel, CamembertTokenizer

In [12]:
from Job_CVs_matching.helper import closest, load_cvs, remove_stopwords, read_pdf_file

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
#Load CVs
cv_embeddings = "/path/to/embeddings/file.txt" # Path to the embeddings file
cvs = load_cvs(cv_embeddings)

In [14]:
model_name = "camembert-base"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
model = CamembertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [17]:
#Parameter of our request file path and number K of recommendation to provide

job_path = "/path/to/the/job.pdf"
K = 30

In [18]:

job_content = read_pdf_file(job_path)

job_no_SW = remove_stopwords(job_content)
job_no_SW_ids = tokenizer.encode(job_no_SW, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

with torch.no_grad():
    output_no_SW = model(job_no_SW_ids)

last_hidden_states = output_no_SW.last_hidden_state

job = last_hidden_states.mean(dim=1)

findings = closest(cvs, job, K)

print(f"The {K} most likely profiles(cvs) to this job are:\n")
findings

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


The 30 most likely profiles(cvs) to this job are:



{'65740': tensor([0.9927]),
 '58356': tensor([0.9926]),
 '35003': tensor([0.9923]),
 '13427': tensor([0.9918]),
 '63706': tensor([0.9916]),
 '7212': tensor([0.9914]),
 '66403': tensor([0.9913]),
 '67437': tensor([0.9910]),
 '71588': tensor([0.9910]),
 '66651': tensor([0.9907]),
 '4953': tensor([0.9907]),
 '60329': tensor([0.9906]),
 '14051': tensor([0.9904]),
 '34391': tensor([0.9904]),
 '66506': tensor([0.9903]),
 '70609': tensor([0.9903]),
 '77369': tensor([0.9903]),
 '47713': tensor([0.9903]),
 '31992': tensor([0.9902]),
 '32493': tensor([0.9900]),
 '34583': tensor([0.9900]),
 '76727': tensor([0.9899]),
 '3177': tensor([0.9899]),
 '5696': tensor([0.9899]),
 '31855': tensor([0.9899]),
 '1908': tensor([0.9897]),
 '71013': tensor([0.9897]),
 '55992': tensor([0.9896]),
 '75745': tensor([0.9896]),
 '60410': tensor([0.9895])}